# "Transformers, GPT-2 and text generation"
> "Stuff You Should Know: some notes for understanding how the Transformer and GPT-2 generates text."

- toc: true
- categories: [transformer, gpt2, huggingface, nlp]
- show_tags: true

## Introduction

Hello! I wanted to share my notes that helped me understand the Transformer, GPT-2 and how they can be used for text generation. This was really helpful because writing it down, and then explaining it in my own words helped me learn so much better.

The primary resources I used are:

* [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
* [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)
* [The Illustrated GPT-2](http://jalammar.github.io/illustrated-gpt2/)
* 🤗 [Hugging Face docs](https://huggingface.co/transformers/)

We will kick things off with the [Transformer](https://arxiv.org/abs/1706.03762), since GPT-2 is just a variant of it.

## Transformer

The Transformer's secret sauce comes from stacking self-attention layers in the encoder-decoder. Attention is exactly what it sounds like, it gives the model the ability to focus on a relevant subset of information. For example, in the sentence below, attention helps the model understand that `one` refers to `ATMs`.

`There are two ATMs in Antarctica and only one of them works.`

So in a nutshell, self-attention allows the model to look at all the other words (also known as **attend to**) in the sentence which lets the Transformer understand how they're all related to the word it is currently processing. 

### Self-attention

Secret sauce recipe:

1. Multiply each word's embedding by three weight matrices to get a query, key and value vector for each word.
2. Next calculate a score for each word so the model knows how much focus to place on each of them. Suppose we are encoding the word `ATMs`. 
3. Take the dot product of the query and key for `ATMs`. Then multiply the query vector of `ATMs` by the key vector of all the other words in the sentence. Now we have a **score for every word as it relates to `ATMs`**.
4. Divide all these scores by a scaling factor that is equal to the square root of the dimension of the key vector. As the dimensionality of the query and key vectors increase, the size of the dot product also increases. To prevent this from blowing up and destabilizing the gradient, we need to rescale these scores. The authors call this **scaled dot-product attention**.
5. Pass these values through softmax so that all the scores sum to 1. At this point, we expect relevant words to have higher scores.
6. Multiply the softmax scores by their respective value vectors so we amplify the high scoring words and dampen the irrelevant ones.
7. Sum up all these values and you get a vector that contains the encoding for `ATMs`.

In practice, we do this calculation with matrices instead of vectors because it is much faster that way. 

![](https://i.imgur.com/CFQz0to.png "created by Javier Abellán")

### Multi-head self-attention

Mapping the relationship between `ATMs` and `one` is just one representation of the sentence (this is learned by one attention head). What if we wanted to know where `one` is? In this case, it would be better for the Transformer to focus on `Antarctica` and `one`, instead of `ATMs`. So the Transformer is equipped with 8 attention heads to learn and capture all these different relationships from different positions, giving it a richer understanding of the sentence. This makes up the multi-head self-attention sub-layer in each encoder-decoder stack.

The way we calculate multi-head self-attention is similar to how we calculate self-attention. We do the same calculation 8 different times, each with their own separate weights to generate a bunch of encodings. Then we concatenate these together, multiply it by another set of weights and you end up with a matrix that contains the encoding of a word from multiple positions. 

![](https://i.imgur.com/Dp8ERiD.png "created by Javier Abellán")

We can see these learned representations with [exBERT](https://exbert.net/) and visualize what each head is doing. So here, notice how head 1 and head 5 (highlighted in blue) are focusing on different parts of the sentence.

![](https://raw.githubusercontent.com/stevhliu/ingolmo/master/images/head1.png)
![](https://raw.githubusercontent.com/stevhliu/ingolmo/master/images/head5.png)

Recent research however suggests that not all of these heads are necessary, and disabling one does not harm the performance of the Transformer because some of the information is redundant ([The Dark Secrets of BERT](https://text-machine-lab.github.io/blog/2020/bert-secrets/#what-types-of-self-attention-patterns-are-learned-and-how-many-of-each-type)).

### Masked multi-head self-attention

But if the Transformer can attend to all the other words, how do we know it's actually learning and not just repeating what it has already seen? The trick is to implement an attention mask that prevents the decoder from seeing what the next word is by setting it to `-inf`. The mask is applied after we've calculated the scores by multiplying the query and key vectors. So now when we take the softmax, the values in the mask go to 0 and the decoder can't attend to those words. The attention mask can be implemented like this (from [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)):

```python
def subsequent_mask(size):
    attn_shape = (1,size,size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0
```

![](https://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png "from The Illustrated GPT-2")

### Model architecture

Now that we understand multi-head and masked multi-head self-attention, it's easier to understand how they fit into the overall architecture of the Transformer. The Transformer stacks 6 encoder-decoders which are pretty much identical except the decoder has an additional masked multi-head self-attention sub-layer. 

![](https://nlp.seas.harvard.edu/images/the-annotated-transformer_14_0.png "from The Annotated Transformer")

#### Encoder 

The encoder has two sub-layers: multi-head self-attention and a fully connected feed-forward network. Between these sub-layers are a residual connection followed by a layer normalization step (check out the [fast.ai](https://course.fast.ai/index.html) course if you need a refresher). The Transformer also adds positional encoding values to the input embeddings because it doesn't know the position of words in the sequence. So positional encodings are a way to explicitly tell the Transformer where words are with respect to one other. 

In brief, the encoder does a bunch of parallelized matrix multiplications (which is why the Transformer is so fast ⚡️) that gets passed off to the encoder above it. By stacking these blocks together, you create a more powerful network with richer representations.

#### Decoder 

The decoder is pretty much identical to the encoder except it has a masked multi-head self-attention sub-layer. The self-attention sub-layer takes the key and values from the top encoder and it gets the query matrix from the masked attention sub-layer below it. Then we just do the same self-attention calculation. The output of each time-step is used by the bottom decoder at the next time-step to generate the next word. Like the encoder, we add positional encodings to indicate a word's location.

#### Linear + Softmax

Lastly, the decoder outputs a vector of numbers that are passed into a linear layer which produces a vector of logits (this is the same size as the model vocabulary). Pass this through a softmax to normalize the values so that they all sum to 1, and then select the highest value to get the predicted word. 

> At each step the model is auto-regressive, consuming the previously generated symbols as additional input when generating the next ([Vaswani et al.](https://arxiv.org/abs/1706.03762)).

## GPT-2

[GPT-2](https://openai.com/blog/better-language-models/#sample8) is a Transformer-based model, but instead of the encoder-decoder architecture, it gets rid of the encoder entirely. It is basically a big stack of decoders with 1.5 billion parameters, a souped-up version of the original GPT which had 110 million parameters. In late May 2020, OpenAI introduced GPT-2's successor, [GPT-3](https://arxiv.org/abs/2005.14165), which is an absolute unit with 175 billion parameters 🤯. The general strategy at OpenAI, when it comes to the GPT family, is to train bigger models on larger datasets. GPT-2 is a causal language model, meaning that it was trained to simply predict the next word in a sentence. The rationale is that if the model is able to predict the next word sufficiently well, this suggests that the model understands the text. For example, consider the sentence:

 `Armadillos can hold their breath for up to six minutes and are known to walk _____.`

You'd probably say `underwater` because why else would it hold it's breath? So in order for GPT-2 to successfully predict `underwater`, it would have to understand that the armadillo is somewhere it can't breathe regularly. In turn, this broad understanding of text carries over to being able to perform well on various NLP tasks (Q&A, summarization, reading comprehension, etc.) without having to fine-tune the model to these tasks. This is known as zero-shot learning, meaning you can just use the model out of the box.

### Model overview

During evaluation, GPT-2 is predicting one word at a time so it would be a waste to redo the self-attention calculations every step for tokens that have already been processed. Instead GPT-2 just keeps the key and value vectors for these tokens so that in the next step, it doesn't need to generate a new query, key and value vector. It simply reuses the ones it had saved from before. 

Let's go through the sentence below to really solidify our understanding of how GPT-2 selects the next word.

`Armadillos can hold their breath for up to six minutes and are known to walk _____`.

1. Create the embedding and positional encodings for `walk`, and multiply that by a weight matrix to produce a concatenated vector of the query, key and values. 
2. Split this into the query, key and value vectors and then reshape these vectors into a matrix. Next, split this matrix up for each attention head.
3. Then we do the usual attention calculation so that we score the current word against all the previous ones. This calculation is happening in each attention head.
4. Once we have all the scores from each attention head, we concatenate them into a single vector. Then we multiply this by another weight matrix that maps the outputs of the attention head into an output vector that we send to the feed-forward sub-layer.
5. The feed-forward network has two layers that the score vector passes through after which we do the usual softmax and get the probabilities over tokens.

![](https://jalammar.github.io/images/gpt2/gpt2-weights-2.png "from The Illustrated GPT-2")

### Byte-level byte-pair encoding (BPE)

Traditional tokenization splits a sentence into individual words, but you could end up with a huge vocabulary resulting in memory issues. If we tokenize each character individually, then we run into the problem of losing some of the semantics of the sentence. As a result, the model doesn't learn the representations as well. Instead, we split the difference and do subword tokenization where we try and cover all the words in our vocabulary by building up the smallest collection of subwords possible. This generally means that the more common words will be tokenized as a whole word, while rarer ones are broken down into smaller chunks. For example, `permutation` has been split into `perm` and `utations` here.

```python
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
print(tokenizer.tokenize('There are more permutations of a standard deck of 52 cards than there are seconds since the Big Bang.'))

['There', 'Ġare', 'Ġmore', 'Ġperm', 'utations', 'Ġof', 'Ġa', 'Ġstandard', 'Ġdeck', 'Ġof', 'Ġ52', 'Ġcards', 'Ġthan', 'Ġthere', 'Ġare', 'Ġseconds', 'Ġsince', 'Ġthe', 'ĠBig', 'ĠBang', '.']
```

This is how the BPE algorithm works:

1. Get the frequency of each word.
2. Create a list of the frequencies of each character (we call them tokens).
3. Merge the most common pair of tokens (also known as byte pairs, hence BPE).
4. Add this pair to your list of tokens and then recount the frequencies for each token.
5. Repeat these steps until you reach your vocabulary size. For GPT-2, this is 50000, so we stop training the tokenizer after 50000 merges.

Anything that the tokenizer hasn't seen before will be tokenized as `<unk>` since it is not in the base vocabulary. 

But GPT-2 takes this a step further by using BPE on the byte sequences themselves 🤯. The characters you see on screen are represented by Unicode code points. For example, the character `H` looks like `U+0048` (we can ignore the `U`). So `0048` is broken down into 4 bytes here. There are 256 bytes available, which becomes the base vocabulary and then these can be merged together to represent any other token in the vocabulary. Now you can tokenize any text without ever needing an `<unk>` token. 

One more thing, the `Ġ` you see from the GPT-2 tokenizer actually represents a whitespace because the encoder doesn't like them, so we need to represent it with some bytes. This is literally from the encoder.py file in OpenAI's GPT-2 🤢:

>And avoids mapping to whitespace/control characters the bpe code barfs on

## Decoding methods 🔍

There are several different decoding methods for generating text, covered in this excellent [post](https://huggingface.co/blog/how-to-generate) by Patrick from Hugging Face, that I will briefly summarize here.

### Greedy search

This method just strings words together based on the next word with the highest probability. But because it only considers the next word, it may miss a word two-steps ahead with an even higher probability. 

### Beam search

Beam search addresses this by checking out a pre-defined number of steps ahead. So if you set `num_beams=3`, it will consider words three-steps ahead and return the sequence with the highest probability. 

But an issue shared by both greedy and beam search is that after a while, it may begin repeating itself. We can try and fix this by introducing a penalty if the models sees the same sequence of words more than once, like `San Francisco`. In this case, we would set the probability to zero the next time the model sees these two words. The downside though is that you'd only be able to mention `San Francisco` once in your generated text.

### Sampling

The next two methods are sampling-based methods, which means randomly picking the next word based on its conditional probability distribution. 

`Cows have best friends and get lonely if they are separated.`

Here, `have` is sampled from the conditioned probability distribution of `Cows`, and `best` is sampled from `Cows`, `have`, and so on. The downside of this approach is that often times you can end up generating nonsense. We can try and make this better by lowering the temperature. This squishes the probability distribution so that you are more likely to pick a high probability word. Notice how if you set the temperature to zero, it essentially becomes a greedy search because you are just picking the next word with the highest probability.

#### Top-*K* sampling

This is the sampling strategy GPT-2 uses. What happens here is that the top *K* most likely words are selected and the probability is redistributed among only these words. But the problem with this method is that it doesn't adapt well to the probability distribution of the next word.

* In the case of a flat distribution, by limiting to only *K* words, we can end up eliminating plausible word candidates which can hinder the models creativity.
* In the case of a sharp distribution, we run the risk of including some words that don't make sense in the sampling pool.

#### Top-*p* sampling

The issue here is that by fixing *K* to a certain number of words, we can't adapt when the next word probability distribution changes. So instead of setting *K*, we can try and set *p* instead, where *p* is the probability. What this means is that we find the set of words whose cumulative probability exceeds *p*. Then we redistribute the probability amongst this set of words. In doing so, the number of words in the next set (as opposed to a fixed *K*) can vary according to the probability distribution.

What this translates to is that when the next word is less predictable, you have a wider range of words to sample from, giving you more options to be creative. When the next word is more predictable, you'll just get less words because it only needs to pick a few words to exceed *p*.